## NOTEBOOK CON DATOS DE PRUEBA PARA ANALIZAR INCIDENCIAS QUE REPORTAN LO MISMO

In [ ]:
# Instalar requirements (en la terminal, activado venv):

# pip install -r requirements.txt

In [3]:
# !pip install seaborn

"pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
from sqlalchemy import create_engine, text

____________________________


### Pruebas SQLite

In [1]:
import sqlite3
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np

In [2]:
import os
os.getcwd()

'c:\\Users\\corre\\OneDrive\\Escritorio\\Desafio\\tripulaciones-data\\data-endpoints'

In [3]:
engine = create_engine("sqlite:///./bbdd/db_test.db")
engine

Engine(sqlite:///./bbdd/db_test.db)

HACEMOS UNA TABLA DE PRUEBA DE INCIDENCIAS

In [4]:
import datetime
df = pd.DataFrame({
    "id": [1,2,3,4],
    "num_tlf": [657283288, 675675675, 456456456, 656834026],
    "date": [datetime.datetime.now(), datetime.datetime(2023, 6, 15), datetime.datetime.now(), datetime.datetime.now()],
    "inc_rec": ["Buenos días, he notado que el ascensor no está funcionando correctamente.",
                "Lamentablemente, parece que hay una tubería rota en el segundo piso.",
                "Me preocupan unas grietas que han aparecido en un pilar del sótano.",
                "Hola, el ascensor no funciona"],
    "categ":["Ascensor", "tupri", "tupri2", np.nan],
    "urg":[np.nan, 3, 1, np.nan]
})
df

,id,num_tlf,date,inc_rec,categ,urg
0,1,657283288,2024-01-17 19:02:22.496844,"Buenos días, he notado que el ascensor no está...",Ascensor,NaN
1,2,675675675,2023-06-15 00:00:00.000000,"Lamentablemente, parece que hay una tubería ro...",tupri,3.0
2,3,456456456,2024-01-17 19:02:22.496844,Me preocupan unas grietas que han aparecido en...,tupri2,1.0
3,4,656834026,2024-01-17 19:02:22.496844,"Hola, el ascensor no funciona",NaN,NaN


LLEVAMOS LA TABLA A LA BASE DE DATOS

In [5]:
df.to_sql("inc_table", con=engine, if_exists="replace")

4

In [6]:
query = "SELECT * FROM inc_table"

pd.read_sql(query, engine)

,index,id,num_tlf,date,inc_rec,categ,urg
0,0,1,657283288,2024-01-17 19:02:22.496844,"Buenos días, he notado que el ascensor no está...",Ascensor,NaN
1,1,2,675675675,2023-06-15 00:00:00.000000,"Lamentablemente, parece que hay una tubería ro...",tupri,3.0
2,2,3,456456456,2024-01-17 19:02:22.496844,Me preocupan unas grietas que han aparecido en...,tupri2,1.0
3,3,4,656834026,2024-01-17 19:02:22.496844,"Hola, el ascensor no funciona",None,NaN


In [7]:
df.columns

Index(['id', 'num_tlf', 'date', 'inc_rec', 'categ', 'urg'], dtype='object')

In [8]:
query = '''SELECT inc_rec FROM inc_table

WHERE (categ IS NULL) OR (urg IS NULL)

'''

result = pd.read_sql(query, engine).values
lista_inc = []

for elem in result:
    lista_inc.append(elem[0])

lista_inc


['Buenos días, he notado que el ascensor no está funcionando correctamente.',
 'Hola, el ascensor no funciona']

In [9]:
for incidencia in lista_inc:
    
    print(incidencia)

Buenos días, he notado que el ascensor no está funcionando correctamente.
Hola, el ascensor no funciona


Hacer un bucle for al código de abajo para clasificar cada incidencia de la lista de arriba.

In [10]:
import re

respuesta_chat = "Categoría: Ascensor, Urgencia: 4"

# Definir patrones de expresiones regulares
patron_categoria = r"Categoría: (\w+)"
patron_urgencia = r"Urgencia: (\d+)"

# Buscar coincidencias en la cadena
match_categoria = re.search(patron_categoria, respuesta_chat)
match_urgencia = re.search(patron_urgencia, respuesta_chat)

# Extraer información y almacenar en variables
categoria = match_categoria.group(1) if match_categoria else None
urgencia = int(match_urgencia.group(1)) if match_urgencia else None


print("Categoría:", categoria)
print("Urgencia:", urgencia)

Categoría: Ascensor
Urgencia: 4


In [11]:
categoria

'Ascensor'

In [12]:
incidencia

'Hola, el ascensor no funciona'

In [13]:
query = """
    UPDATE inc_table
    SET categ = :categoria, urg = :urgencia
    WHERE inc_rec = :incidencia
"""

# Ejecuta la expresión de actualización
with engine.connect() as connection:
    result = connection.execute(text(query), {'categoria': categoria, 'urgencia': urgencia, 'incidencia': incidencia})
    # connection.commit()
    connection.connection.commit()
    print(f"Número de filas afectadas: {result.rowcount}")

Número de filas afectadas: 1


In [14]:
df["inc_rec"][3] == incidencia

True

________________

***LA TABLA***

In [20]:
query = "SELECT * FROM inc_table"

pd.read_sql(query, engine)

,index,id,num_tlf,date,inc_rec,categ,urg
0,0,1,657283288,2024-01-17 19:02:22.496844,"Buenos días, he notado que el ascensor no está...",Ascensor,NaN
1,1,2,675675675,2023-06-15 00:00:00.000000,"Lamentablemente, parece que hay una tubería ro...",tupri,3.0
2,2,3,456456456,2024-01-17 19:02:22.496844,Me preocupan unas grietas que han aparecido en...,tupri2,1.0
3,3,4,656834026,2024-01-17 19:02:22.496844,"Hola, el ascensor no funciona",Ascensor,4.0


### COMPROBACION DE QUE DOS PERSONAS O MAS ESTEN REPORTANDO EL MISMO PROBLEMA


Con esta query cogemos los numeros de telefono que tengan una incidencia con la misma categoria y fecha y luego miramos su ubicación


In [19]:
query = """
        SELECT Dirección 
FROM info_table
WHERE Teléfono IN (SELECT num_tlf 
                  FROM inc_table
                  WHERE DATE(date) = (SELECT DATE(MAX(date)) FROM inc_table)
                    AND categ = (SELECT categ FROM inc_table ORDER BY date DESC LIMIT 1));

        """

direcciones_tlf=pd.read_sql(query, engine)
direcciones_tlf

,Dirección
0,"Plaza Falsa 67, 46002 Valencia, España"
1,"Plaza Falsa 67, 46002 Valencia, España"


In [156]:
# Comprobamos que los dos numeros de tlf esten en la misma direccion
direcciones_tlf['Dirección'][0]==direcciones_tlf['Dirección'][1]

True

Y AL VER QUE ESTO ES TRUE HABRIA QUE HACER UN CODIGO QUE MUESTRE EN LA APLICACION SOLO EL PRIMER MENSAJE